### Predicting formation enthalpies for solid solutions of Lanthanides Orthophosphates


In [1]:
import os
import sys
import inspect

print(sys.version)

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
grandparentdir = os.path.dirname(parentdir)

sys.path.insert(0, grandparentdir) 

3.9.6 (default, Nov 10 2023, 13:38:27) 
[Clang 15.0.0 (clang-1500.1.0.2.5)]


In [2]:
import read_data
import featureSpan
import lasso
import Utils
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import scipy.optimize
import pandas as pd
import seaborn as sns
import scipy.stats as ss
from sklearn.linear_model import Lasso
from itertools import combinations, product
import itertools
import math
import pandas as pd

## Monazite

#### Load data from files

In [3]:
list1 = read_data.readData("../../data/DATA_HE_monazite.dat")
print("Shape of List1 is {}".format(list1.shape))
print(list1)

Shape of List1 is (525, 4)
[[5.70000000e+01 5.80000000e+01 7.50000000e-01 1.29717527e-01]
 [5.70000000e+01 5.80000000e+01 6.25000000e-01 1.90974365e-01]
 [5.70000000e+01 5.80000000e+01 5.00000000e-01 1.92712564e-01]
 ...
 [7.00000000e+01 7.10000000e+01 5.00000000e-01 2.12444086e-02]
 [7.00000000e+01 7.10000000e+01 3.75000000e-01 2.22639048e-02]
 [7.00000000e+01 7.10000000e+01 2.50000000e-01 2.09098752e-02]]


In [4]:
list2 = read_data.readCSVData("../../data/Data_Ln-monazite.csv", Volume=False)
print("Shape of List2 is {}".format(list2.shape))
print((list2))

Shape of List2 is (15, 8)
[[ 57.      138.90547  19.1773  139.1      11.        1.1      49.95
    1.216  ]
 [ 58.      140.116    20.198   146.5      12.        1.12     36.758
    1.196  ]
 [ 59.      140.90765  21.624   150.1      12.3       1.13     38.98
    1.179  ]
 [ 60.      144.242    22.1     154.1      12.95      1.14     40.41
    1.163  ]
 [ 61.      145.       22.3     157.4      13.6       1.13     41.1
    1.144  ]
 [ 62.      150.36     23.4     160.3      14.25      1.17     41.4
    1.132  ]
 [ 63.      151.964    24.92    163.1      14.9       1.2      42.7
    1.12   ]
 [ 64.      157.25     20.63    165.2      15.9       1.2      44.
    1.107  ]
 [ 65.      158.92535  21.91    164.5      16.2       1.1      39.79
    1.095  ]
 [ 66.      162.5      22.8     165.4      16.85      1.22     41.47
    1.083  ]
 [ 67.      164.93032  22.84    167.3      17.5       1.23     42.5
    1.072  ]
 [ 68.      167.259    22.74    169.1      18.15      1.24     42.7
    1.062

#### Generated Elemental Training/Testing sets from loaded data

In [5]:
def generateFeatures_full2(HEList, featureList):
    X = []
    Y = []
    
    for i in range(len(HEList)):
        Y.append(HEList[i][3])
        
        x = HEList[i][2]
        X1 = featureList[int(HEList[i][0]-57)]
        X2 = featureList[int(HEList[i][1]-57)]

        #X.append(np.concatenate((np.array([x, 1/x, x * x, 1 / (x * x), 1-x, 1/(1-x), (1-x) * (1-x), 1/( (1-x) * (1-x))]), np.array([abs(X1-X2)/2.0, 2.0 / (abs(X1-X2)), (X1 + X2) / 2.0, 2.0 / (X1 + X2)]).flatten())))
        mfeatures = np.array([x, 1/x, x * x, 1 / (x * x), 1-x, 1/(1-x), (1-x) * (1-x), 1/( (1-x) * (1-x))])
        
        f1 = []
        f2 = []
        f3 = []
        f4 = []
        
        for j in range(len(X1)-3):
            f1.append([abs(X1[j]-X2[j])/2.0])
            f2.append([2.0/abs(X1[j]-X2[j])])
            f3.append([(X1[j] + X2[j]) / 2.0])
            f4.append([2.0 / (X1[j] + X2[j])])
            
        #rint(f3)
        #print(np.asarray(f3).flatten())
    
        #print(np.concatenate((mfeatures, np.asarray(f1).flatten(), np.asarray(f3).flatten())))
              
        f5 = []
        f6 = []
        f7 = []
        
        for j in range(5,7):
            f5.append([abs(X1[j]-X2[j])/2.0])
            f6.append([(X1[j] + X2[j]) / 2.0])
            f7.append([2.0 / (X1[j] + X2[j])])            
            
        f8 = []
        f9 = []
        f10 = []
        f11 = []
        f12 = []
        f13 = []
        f14 = []
        f15 = []
        f16 = []
        f17 = []
        f18 = []
        f19 = []
        
        for j in range(7,len(X1)):
            f8.append([abs(X1[j]-X2[j])/2.0])
            f9.append([2.0/abs(X1[j]-X2[j])])
            f10.append([(X1[j] + X2[j]) / 2.0])
            f11.append([2.0 / (X1[j] + X2[j])])
            f12.append([pow(abs(X1[j]-X2[j])/2.0,2)])
            f13.append([pow(2.0/abs(X1[j]-X2[j]),2)])
            f14.append([pow((X1[j] + X2[j]) / 2.0,2)])
            f15.append([pow(2.0 / (X1[j] + X2[j]),2)])
            f16.append([pow(abs(X1[j]-X2[j])/2.0,3)])
            f17.append([pow(2.0/abs(X1[j]-X2[j]),3)])
            f18.append([pow((X1[j] + X2[j]) / 2.0,3)])
            f19.append([pow(2.0 / (X1[j] + X2[j]),3)])
            
        X.append(np.concatenate((mfeatures, np.asarray(f1).flatten(),  np.asarray(f2).flatten(),  np.asarray(f3).flatten(), np.asarray(f4).flatten(), np.asarray(f5).flatten(), np.asarray(f6).flatten(),np.asarray(f7).flatten(),np.asarray(f8).flatten(),np.asarray(f9).flatten(),np.asarray(f10).flatten(),np.asarray(f11).flatten(),np.asarray(f12).flatten(),np.asarray(f13).flatten(),np.asarray(f14).flatten(),np.asarray(f15).flatten(),np.asarray(f16).flatten(),np.asarray(f17).flatten(),np.asarray(f18).flatten(),np.asarray(f19).flatten()     )))
            
    return X, Y

In [6]:
listX, listY = generateFeatures_full2(list1, list2)
X = np.asarray(listX)
Y = np.asarray(listY)
print(X.shape)
print(Y.shape)

elemental_list = ["m", "1/m", "m^2", "(1/m)^2", "(1-m)", "1/(1-m)", "(1-m)^2", "(1/(1-m))^2", "[AD:Z]", "[AD:M]", "[AD:IP2]", "[AD:Young]", "[AD:Charge]",  "(1/[AD:Z])", "(1/[AD:M])", "(1/[AD:IP2])", "(1/[AD:Young])", "(1/[AD:Charge])",  "[AM:Z]", "[AM:M]", "[AM:IP2]", "[AM:Young]", "[AM:Charge]",  "(1/[AM:Z])", "(1/[AM:M])", "(1/[AM:IP2])", "(1/[AM:Young])", "(1/[AM:Charge])", "[AD:electronegativity]", "[AD:IP3]", "[AM:electronegativity]", "[AM:IP3]","(1/[AM:electronegativity])", "(1/[AM:IP3])" ,    "[AD:R]", "(1/[AD:R])","[AM:R]","(1/[AM:R])",   "([AD:R])^2", "(1/[AD:R])^2" , "([AM:R])^2","(1/[AM:R])^2",    "([AD:R])^3", "(1/[AD:R])^3", "([AM:R])^3", "(1/[AM:R])^3"         ]
print(len(elemental_list))
print(elemental_list)
m, n = X.shape

elemental_features = []

for i in elemental_list:
    elemental_features.append("("+i+")")
        
print("The elemental features are: \n {}".format(elemental_features))

(525, 46)
(525,)
46
['m', '1/m', 'm^2', '(1/m)^2', '(1-m)', '1/(1-m)', '(1-m)^2', '(1/(1-m))^2', '[AD:Z]', '[AD:M]', '[AD:IP2]', '[AD:Young]', '[AD:Charge]', '(1/[AD:Z])', '(1/[AD:M])', '(1/[AD:IP2])', '(1/[AD:Young])', '(1/[AD:Charge])', '[AM:Z]', '[AM:M]', '[AM:IP2]', '[AM:Young]', '[AM:Charge]', '(1/[AM:Z])', '(1/[AM:M])', '(1/[AM:IP2])', '(1/[AM:Young])', '(1/[AM:Charge])', '[AD:electronegativity]', '[AD:IP3]', '[AM:electronegativity]', '[AM:IP3]', '(1/[AM:electronegativity])', '(1/[AM:IP3])', '[AD:R]', '(1/[AD:R])', '[AM:R]', '(1/[AM:R])', '([AD:R])^2', '(1/[AD:R])^2', '([AM:R])^2', '(1/[AM:R])^2', '([AD:R])^3', '(1/[AD:R])^3', '([AM:R])^3', '(1/[AM:R])^3']
The elemental features are: 
 ['(m)', '(1/m)', '(m^2)', '((1/m)^2)', '((1-m))', '(1/(1-m))', '((1-m)^2)', '((1/(1-m))^2)', '([AD:Z])', '([AD:M])', '([AD:IP2])', '([AD:Young])', '([AD:Charge])', '((1/[AD:Z]))', '((1/[AD:M]))', '((1/[AD:IP2]))', '((1/[AD:Young]))', '((1/[AD:Charge]))', '([AM:Z])', '([AM:M])', '([AM:IP2])', '([AM:

In [7]:
dfX = pd.DataFrame(data=X, columns=elemental_features)

new_features=[]
new_columns = []

for i in range(n):
    for j in range(i):
        new_features.append(elemental_features[i]+"*"+elemental_features[j])
        new_columns.append(dfX[elemental_features[i]].values * dfX[elemental_features[j]].values)


for i in range(n):
    for j in range(i):
        for k in range(j):
            new_features.append(elemental_features[i]+"*"+elemental_features[j]+"*"+elemental_features[k])
            new_columns.append(dfX[elemental_features[i]] * dfX[elemental_features[j]] * dfX[elemental_features[k]])
            
            
new_columns = np.asarray(new_columns)
dfX = pd.concat(
    [
        dfX,
        pd.DataFrame(
            new_columns.T, 
            index=dfX.index, 
            columns=new_features
        )
    ], axis=1
)

dfX.head()

dfX.std() == 0
dfX=dfX.loc[:, dfX.std() > 0]
dfX.head()

,(m),(1/m),(m^2),((1/m)^2),((1-m)),(1/(1-m)),((1-m)^2),((1/(1-m))^2),([AD:Z]),([AD:M]),...,((1/[AM:R])^3)*(([AM:R])^3)*([AD:R]),((1/[AM:R])^3)*(([AM:R])^3)*((1/[AD:R])),((1/[AM:R])^3)*(([AM:R])^3)*([AM:R]),((1/[AM:R])^3)*(([AM:R])^3)*((1/[AM:R])),((1/[AM:R])^3)*(([AM:R])^3)*(([AD:R])^2),((1/[AM:R])^3)*(([AM:R])^3)*((1/[AD:R])^2),((1/[AM:R])^3)*(([AM:R])^3)*(([AM:R])^2),((1/[AM:R])^3)*(([AM:R])^3)*((1/[AM:R])^2),((1/[AM:R])^3)*(([AM:R])^3)*(([AD:R])^3),((1/[AM:R])^3)*(([AM:R])^3)*((1/[AD:R])^3)
0,0.750,1.333333,0.562500,1.777778,0.250,4.000000,0.062500,16.000000,0.5,0.605265,...,0.01,100.0,1.206,0.829187,0.0001,10000.0,1.454436,0.687552,0.000001,1000000.0
1,0.625,1.600000,0.390625,2.560000,0.375,2.666667,0.140625,7.111111,0.5,0.605265,...,0.01,100.0,1.206,0.829187,0.0001,10000.0,1.454436,0.687552,0.000001,1000000.0
2,0.500,2.000000,0.250000,4.000000,0.500,2.000000,0.250000,4.000000,0.5,0.605265,...,0.01,100.0,1.206,0.829187,0.0001,10000.0,1.454436,0.687552,0.000001,1000000.0
3,0.375,2.666667,0.140625,7.111111,0.625,1.600000,0.390625,2.560000,0.5,0.605265,...,0.01,100.0,1.206,0.829187,0.0001,10000.0,1.454436,0.687552,0.000001,1000000.0
4,0.250,4.000000,0.062500,16.000000,0.750,1.333333,0.562500,1.777778,0.5,0.605265,...,0.01,100.0,1.206,0.829187,0.0001,10000.0,1.454436,0.687552,0.000001,1000000.0


In [8]:
print(len(dfX.columns.values))
np.array(dfX.columns.values)

16255


array(['(m)', '(1/m)', '(m^2)', ...,
       '((1/[AM:R])^3)*(([AM:R])^3)*((1/[AM:R])^2)',
       '((1/[AM:R])^3)*(([AM:R])^3)*(([AD:R])^3)',
       '((1/[AM:R])^3)*(([AM:R])^3)*((1/[AD:R])^3)'], dtype=object)

In [9]:
def LassoFit(lmb, X, Y, max_iter=100000, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
    lasso =  Lasso(alpha=lmb, max_iter=max_iter)
    lasso.fit(X_standardized, Y.copy())
    coef =  lasso.coef_
    selected_indices = coef.nonzero()[0]
    selected_features = np.array(dfX.columns.values)[selected_indices]
    Y_predict = lasso.predict(X_standardized)
    MAE, MSE, ME = Utils.compute_error(Y.copy(), Y_predict)
        
    return coef, selected_indices, selected_features, MAE, MSE, ME

LassoFit(0.01, dfX, Y)

(array([ 0., -0.,  0., ..., -0.,  0.,  0.]),
 array([  556,  1257,  4309,  4392,  4733,  5117,  6030,  7047,  7370,
         7447,  7476,  9528,  9658,  9887,  9899,  9905, 12570, 12572,
        12657, 12872, 12923, 15282]),
 array(['((1/[AM:electronegativity]))*([AM:IP2])',
        '([AD:Young])*((1-m)^2)*(m^2)',
        '((1/[AM:Charge]))*((1/[AM:IP2]))*([AD:M])',
        '([AD:electronegativity])*([AD:M])*(1/(1-m))',
        '([AD:IP3])*((1/m)^2)*(1/m)',
        '([AD:IP3])*([AD:electronegativity])*([AD:IP2])',
        '([AM:IP3])*([AM:electronegativity])*((1/[AM:IP2]))',
        '((1/[AM:IP3]))*((1/[AM:electronegativity]))*([AM:IP2])',
        '([AD:R])*((1/[AM:IP2]))*([AD:Young])',
        '([AD:R])*([AD:electronegativity])*([AD:IP2])',
        '([AD:R])*([AD:IP3])*([AD:Young])', '(([AD:R])^2)*((1-m)^2)*(m^2)',
        '(([AD:R])^2)*((1/[AD:Charge]))*([AD:Young])',
        '(([AD:R])^2)*((1/[AM:Charge]))*((1/[AM:IP2]))',
        '(([AD:R])^2)*([AD:electronegativity])*([AD:IP2])',


In [10]:
#### Define a function which fits Lasso to have no more nonzero coefficients than a given threshold 
def LassoSelect(X, Y, min, max, step, threshold, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
        
    found = False
    for lmbda in np.arange (min, max, step):
        coef, selected_indices, selected_features, MAE, MSE, ME = LassoFit(lmbda, X.copy(), Y.copy())
        if len(selected_indices) <= threshold:
            found = True
            break
    
    if found:
        print("FOUND with threshold: {}".format(threshold))
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
            
    else:
        print("NOT FOUND with threshold: {}".format(threshold))
        print("Closest are: ")
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
     
    X_reduced = X[selected_features]
    
    return X_reduced


X_reduced = LassoSelect(dfX, Y, 0.001, 0.101, 0.005, 30)
X_reduced.head()

FOUND with threshold: 30
Lambda: 0.011, nnz: 22, MAE: 0.07719329268647325, MSE: 0.013123008262563973, MAPE: 0.16152982224122478


,([AD:Young])*((1-m)^2)*(m^2),((1/[AM:IP2]))*([AD:M])*(1/(1-m)),((1/[AM:Charge]))*((1/[AM:IP2]))*([AD:M]),([AD:electronegativity])*([AD:M])*(1/(1-m)),([AD:IP3])*((1/m)^2)*(1/m),([AD:IP3])*([AD:electronegativity])*([AD:IP2]),([AM:IP3])*([AM:electronegativity])*((1/[AM:IP2])),((1/[AM:IP3]))*((1/[AM:electronegativity]))*([AM:IP2]),([AD:R])*((1/[AM:IP2]))*([AD:Young]),([AD:R])*([AD:electronegativity])*([AD:IP2]),...,(([AD:R])^2)*((1/[AD:Charge]))*([AD:Young]),(([AD:R])^2)*((1/[AM:Charge]))*((1/[AM:IP2])),(([AD:R])^2)*([AD:electronegativity])*([AD:IP2]),(([AD:R])^2)*([AD:electronegativity])*((1/[AD:Young])),(([AD:R])^3)*((1-m)^2)*(m),(([AD:R])^3)*((1-m)^2)*(m^2),(([AD:R])^3)*((1/[AD:M]))*([AD:Young]),(([AD:R])^3)*((1/[AM:IP2]))*((1/[AD:Charge])),(([AD:R])^3)*((1/[AM:Charge]))*((1/[AD:Charge])),((1/[AM:R])^3)*((1-m)^2)*(m^2)
0,0.130078,0.122974,0.002673,0.024211,15.634963,0.033663,2.444321,0.409112,0.001879,0.000051,...,0.00074,4.416806e-07,5.103500e-07,2.702703e-07,4.687500e-08,3.515625e-08,0.000006,1.015865e-07,1.739130e-07,0.020043
1,0.203247,0.081982,0.002673,0.016140,27.017216,0.033663,2.444321,0.409112,0.001879,0.000051,...,0.00074,4.416806e-07,5.103500e-07,2.702703e-07,8.789063e-08,5.493164e-08,0.000006,1.015865e-07,1.739130e-07,0.031317
2,0.231250,0.061487,0.002673,0.012105,52.768000,0.033663,2.444321,0.409112,0.001879,0.000051,...,0.00074,4.416806e-07,5.103500e-07,2.702703e-07,1.250000e-07,6.250000e-08,0.000006,1.015865e-07,1.739130e-07,0.035632
3,0.203247,0.049189,0.002673,0.009684,125.079704,0.033663,2.444321,0.409112,0.001879,0.000051,...,0.00074,4.416806e-07,5.103500e-07,2.702703e-07,1.464844e-07,5.493164e-08,0.000006,1.015865e-07,1.739130e-07,0.031317
4,0.130078,0.040991,0.002673,0.008070,422.144000,0.033663,2.444321,0.409112,0.001879,0.000051,...,0.00074,4.416806e-07,5.103500e-07,2.702703e-07,1.406250e-07,3.515625e-08,0.000006,1.015865e-07,1.739130e-07,0.020043


In [11]:
def LassoL0(X, Y, nnz):    
    nr, nc = X.shape
    X = np.column_stack((X, np.ones(nr)))
    se_min = np.inner(Y, Y)
    coef_min, permu_min = None, None
    for permu in combinations(range(nc), nnz):
        X_ls = X[:, permu + (-1,)]
        coef, se, __1, __2 = np.linalg.lstsq(X_ls, Y, rcond=-1)
        try:
            if se[0] < se_min: 
                se_min = se[0]
                coef_min, permu_min = coef, permu
        except:
            pass
        
    return coef_min, permu_min

In [12]:
def LassoL0Fit(X_reduced, Y, nnz, log=True):
    
    scaler = StandardScaler()
    scaler.fit(X_reduced)
    X_std = scaler.transform(X_reduced)
    
    nr, nc = X_reduced.shape
   
    coefficients, selected_indices = LassoL0(X_std, Y, nnz)
   
    coefficients = np.array(coefficients)
    selected_indices = np.array(selected_indices)
    feature_reduced = np.array(X_reduced.columns.values)
    feature_list_selected = feature_reduced[selected_indices]
    
    X_selected = X_reduced[feature_list_selected]

    mean_selected = X_selected.mean()
    std_selected = X_selected.std()

    
    if log:
        print("Lasso: selected coefficients are: {}".format(coefficients))
        print("Lasso: selected features are: {}".format(feature_list_selected))
        
    #-mean/std
    mean_std = []
    for i in range(len(selected_indices)):
        mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
 
    sum_mean_std = sum(mean_std)

    for i in range(len(selected_indices)):
        coefficients[i] = coefficients[i] / std_selected[i]

    
    coefficients[len(selected_indices)] -= sum_mean_std
    
    function = str(coefficients[0])+" * "+feature_list_selected[0]
    
    for i in range(1, len(selected_indices)):
        if coefficients[i] >= 0:
            function += " + " + str(coefficients[i])+" * "+feature_list_selected[i]
        else:
            function += " - " + str(abs(coefficients[i]))+" * "+feature_list_selected[i]

    
    if coefficients[len(selected_indices)] >= 0:
        function += " + " + str(coefficients[len(selected_indices)])
    else:
        function += " - " + str(abs(coefficients[len(selected_indices)]))
    
    if log:
        print("Constructed function is: {}".format(function))

    X_selected = np.column_stack((X_selected, np.ones(X_selected.shape[0])))
    Y_predict = X_selected[:,0] * coefficients[0]

    for i in range(1,len(selected_indices)+1):
        Y_predict = Y_predict + X_selected[:,i] * coefficients[i]
    
    if log:
        Utils.print_error(Y.copy(),Y_predict,"Lasso L0: {} coef".format(nnz))
    
    return Y_predict, coefficients, selected_indices

In [13]:
LassoL0Fit(X_reduced, Y.copy(), 1);

Lasso: selected coefficients are: [1.9258487  1.65053546]
Lasso: selected features are: ['(([AD:R])^2)*((1-m)^2)*(m^2)']
Constructed function is: 19964.397012275604 * (([AD:R])^2)*((1-m)^2)*(m^2) + 0.03823005580687311
Lasso L0: 1 coef
Mean absolute error: 0.23440353500934025
Mean squared error: 0.12966173716925883
Mean absolute percentage error: 0.47722992791075874


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18328/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18328/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [14]:
LassoL0Fit(X_reduced, Y.copy(), 2);

Lasso: selected coefficients are: [1.19213778 0.80645493 1.65053546]
Lasso: selected features are: ['(([AD:R])^2)*((1-m)^2)*(m^2)'
 '(([AD:R])^2)*((1/[AD:Charge]))*([AD:Young])']
Constructed function is: 12358.349813586337 * (([AD:R])^2)*((1-m)^2)*(m^2) + 113.70014438974141 * (([AD:R])^2)*((1/[AD:Charge]))*([AD:Young]) - 0.00027081991228472546
Lasso L0: 2 coef
Mean absolute error: 0.0858139919339167
Mean squared error: 0.01762399568696247
Mean absolute percentage error: 0.15132899915312942


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18328/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18328/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [15]:
LassoL0Fit(X_reduced, Y.copy(), 3);

Lasso: selected coefficients are: [1.12888676 0.82458208 0.06760148 1.65053546]
Lasso: selected features are: ['(([AD:R])^2)*((1-m)^2)*(m^2)'
 '(([AD:R])^2)*((1/[AD:Charge]))*([AD:Young])'
 '(([AD:R])^2)*([AD:electronegativity])*((1/[AD:Young]))']
Constructed function is: 11702.655275257646 * (([AD:R])^2)*((1-m)^2)*(m^2) + 116.25584672044813 * (([AD:R])^2)*((1/[AD:Charge]))*([AD:Young]) + 6851.101184992974 * (([AD:R])^2)*([AD:electronegativity])*((1/[AD:Young])) - 0.027897503485826425
Lasso L0: 3 coef
Mean absolute error: 0.08378926713900582
Mean squared error: 0.015297046877808473
Mean absolute percentage error: 0.16991995661729958


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18328/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18328/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [16]:
LassoL0Fit(X_reduced, Y.copy(), 4);

Lasso: selected coefficients are: [0.03047906 1.14457166 0.80743666 0.05511705 1.65053546]
Lasso: selected features are: ['([AM:IP3])*([AM:electronegativity])*((1/[AM:IP2]))'
 '(([AD:R])^2)*((1-m)^2)*(m^2)'
 '(([AD:R])^2)*((1/[AD:Charge]))*([AD:Young])'
 '(([AD:R])^2)*([AD:electronegativity])*((1/[AD:Young]))']
Constructed function is: 0.16846683350417863 * ([AM:IP3])*([AM:electronegativity])*((1/[AM:IP2])) + 11865.25351239687 * (([AD:R])^2)*((1-m)^2)*(m^2) + 113.83855565424827 * (([AD:R])^2)*((1/[AD:Charge]))*([AD:Young]) + 5585.86140804811 * (([AD:R])^2)*([AD:electronegativity])*((1/[AD:Young])) - 0.39040786421309637
Lasso L0: 4 coef
Mean absolute error: 0.080719074338445
Mean squared error: 0.01455332251900922
Mean absolute percentage error: 0.16621032746638392


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18328/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18328/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [17]:
LassoL0Fit(X_reduced, Y.copy(), 5);

Lasso: selected coefficients are: [0.13254462 0.04760551 0.5539126  0.96138746 0.32153573 1.65053546]
Lasso: selected features are: ['([AD:Young])*((1-m)^2)*(m^2)'
 '([AD:IP3])*([AD:electronegativity])*([AD:IP2])'
 '([AD:R])*((1/[AM:IP2]))*([AD:Young])' '(([AD:R])^2)*((1-m)^2)*(m^2)'
 '(([AD:R])^3)*((1/[AM:IP2]))*((1/[AD:Charge]))']
Constructed function is: 0.6122066603688889 * ([AD:Young])*((1-m)^2)*(m^2) + 0.40855478510492305 * ([AD:IP3])*([AD:electronegativity])*([AD:IP2]) + 34.84687727390977 * ([AD:R])*((1/[AM:IP2]))*([AD:Young]) + 9966.266247427207 * (([AD:R])^2)*((1-m)^2)*(m^2) + 175670.77173944164 * (([AD:R])^3)*((1/[AM:IP2]))*((1/[AD:Charge])) - 0.06426640736829148
Lasso L0: 5 coef
Mean absolute error: 0.08105715442403873
Mean squared error: 0.014014528175901875
Mean absolute percentage error: 0.2284482265307172


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18328/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18328/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]
